In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

#!pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl

1.11.0+cu113
     |████████████████████████████████| 7.9 MB 11.8 MB/s 
     |████████████████████████████████| 3.5 MB 10.2 MB/s 


In [3]:
import torch
import pickle
import random
import numpy as np
import math
from torch_geometric.data import DenseDataLoader

from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GATConv, BatchNorm, GraphNorm, GCNConv, DenseGraphConv, DenseSAGEConv
from torch_geometric.nn import global_mean_pool, dense_diff_pool, dense_mincut_pool
from torch_geometric.utils import to_dense_adj, to_dense_batch
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()
print(device)


cuda:0


In [4]:
with open(r'/content/drive/MyDrive/Augmented_Elliptic/address_level/address_subgraphs.pkl', "rb") as input_file:
    dataset = pickle.load(input_file)


In [5]:
print('====================')
print(f'Number of graphs: {len(dataset)}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Number of graphs: 102077

Data(edge_index=[2, 6060], interactions=[6060], num_nodes=98, x=[98, 25], y=node
2397173837    0
Name: class, dtype: int64, edge_attr=[6060, 4])
Number of nodes: 98
Number of edges: 6060
Average node degree: 61.84
Has isolated nodes: False
Has self-loops: True
Is undirected: False


In [6]:
classes = {dataset[i].y.index.map(str)[0]: dataset[i].y.item() for i in range(len(dataset))}
classes_1 = dict(filter(lambda i: i[1] == 1, classes.items()))
classes_0 = dict(filter(lambda i: i[1] == 0, classes.items()))

print(f'Number of high risk addresses: {len(classes_1.keys())}')
print(f'Number of safe addresses: {len(classes_0.keys())}')

# Balance the high risk and safe addresses (undersampling safe addresses)
random.seed(1993)
sample_safe = random.sample(list(classes_0.keys()), k = len(classes_1.keys()))

classes_0_trim = dict(filter(lambda i: i[0] in sample_safe, classes_0.items()))
classes_balanced = {**classes_1, **classes_0_trim}

Number of high risk addresses: 10169
Number of safe addresses: 91579


Need a way to get high risk graphs and then train against balanced number of licit graphs

In [7]:
classes_balanced_keys = list(classes_balanced.keys())
train_share = 0.8

train_keys = random.sample(classes_balanced_keys, k = round(train_share*len(classes_balanced_keys)))
test_keys = random.sample(classes_balanced_keys, k = len(classes_balanced_keys) - round(train_share*len(classes_balanced_keys)))

train_dataset = [graph for graph in dataset if graph.y.index[0] in train_keys] 
test_dataset = [graph for graph in dataset if graph.y.index[0] in test_keys]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of testing graphs: {len(test_dataset)}')

Number of training graphs: 16270
Number of testing graphs: 4068


In [8]:
for data in train_dataset:
  data.y = torch.tensor(data.y).type(torch.LongTensor)

for data in test_dataset:
  data.y = torch.tensor(data.y).type(torch.LongTensor) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [9]:
from torch_geometric.loader import DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
'''
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
'''

"\nfor step, data in enumerate(train_loader):\n    print(f'Step {step + 1}:')\n    print('=======')\n    print(f'Number of graphs in the current batch: {data.num_graphs}')\n    print(data)\n    print()\n"

In [10]:
# GAT

from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GraphNorm, GATConv, GATv2Conv, SAGPooling
from torch_geometric.nn import global_mean_pool, GraphMultisetTransformer
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, roc_auc_score


class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, att_heads):
        super(GAT, self).__init__()
        torch.manual_seed(12345)
        
        self.gat_node_1 = GATv2Conv(25, hidden_channels, edge_dim = 4, heads = att_heads)
        self.mincut1 = MinCut(in_channels = hidden_channels * att_heads, hidden_channels = hidden_channels, out_channels = out_channels)
        self.graphnorm1 = GraphNorm(out_channels)
        
        #self.gat_node_2 = GATv2Conv(hidden_channels * att_heads, hidden_channels, edge_dim = 4, heads = att_heads)
        #self.graphnorm2 = GraphNorm(out_channels)
        
        #self.gat_node_3 = GATv2Conv(hidden_channels * att_heads, out_channels, edge_dim = 4, heads = att_heads)
        #self.gat_node_3 = GATv2Conv(hidden_channels * att_heads, hidden_channels, edge_dim = 4, heads = att_heads)
        
        #self.graphnorm3 = GraphNorm(out_channels * att_heads)
        #self.graphnorm3 = GraphNorm(hidden_channels * att_heads)
        
    
        #self.lin = Linear(out_channels * att_heads, 2)
        self.lin = Linear(out_channels, 2)

    def forward(self, x, edge_index, edge_attr, batch):
        # 1. Obtain node embeddings 
        x = x.float()

        x = self.gat_node_1(x, edge_index, edge_attr = edge_attr)
        x = F.leaky_relu(x)
        x = self.mincut1(x, edge_index, batch)
        x = self.graphnorm1(x)
        #x = self.gat_node_2(x, edge_index, edge_attr = edge_attr)
        #x = F.dropout(x, p=0.3, training=self.training)
        #x = F.leaky_relu(x)
        #x = self.mincut1(x, edge_index, batch)
        #x = self.graphnorm2(x)
        #x, weights = self.gat_node_3(x, edge_index, edge_attr = edge_attr, return_attention_weights = True)
        #x = F.dropout(x, p=0.3, training=self.training)
        #x = F.leaky_relu(x)


        # 2. Readout layer - turns node and edge embeddings into a graph embedding
        #x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        

        # 3. Apply a final classifier
        x = self.lin(x)
        x = F.softmax(x, dim = -1)
        
        return x



In [11]:
# MinCut
class MinCut(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=32):
        super().__init__()

        self.conv1 = GCNConv(in_channels, hidden_channels)
        num_nodes = math.ceil(0.5 * average_nodes)
        self.pool1 = Linear(hidden_channels, num_nodes)

        self.conv2 = DenseGraphConv(hidden_channels, hidden_channels)
        num_nodes = math.ceil(0.5 * num_nodes)
        self.pool2 = Linear(hidden_channels, num_nodes)

        self.conv3 = DenseGraphConv(hidden_channels, hidden_channels)

        self.lin1 = Linear(hidden_channels, out_channels)
        #self.lin2 = Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index).relu()

        x, mask = to_dense_batch(x, batch)
        adj = to_dense_adj(edge_index, batch)

        s = self.pool1(x)
        x, adj, mc1, o1 = dense_mincut_pool(x, adj, s, mask)

        x = self.conv2(x, adj).relu()
        s = self.pool2(x)

        x, adj, mc2, o2 = dense_mincut_pool(x, adj, s)

        x = self.conv3(x, adj)

        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        #x = self.lin2(x)
        #return F.log_softmax(x, dim=-1), mc1 + mc2, o1 + o2
        return x


In [16]:
train_loader.dataset[0].num_nodes

4

In [17]:
average_nodes = 0
total_data = 0

for data in train_loader:
    total_data += 1
    average_nodes = average_nodes + data.num_nodes

average_nodes = average_nodes / total_data
print(total_data)
print(average_nodes)
#average_nodes = 3000

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GAT(hidden_channels = 128, out_channels = 32, att_heads = 10).to(device)
#model = DiffPool().to(device)
#model = MinCut(in_channels = 25, hidden_channels = 256, out_channels = 2).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-4)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = torch.nn.CrossEntropyLoss()
print(model)

509
2040.6699410609037
GAT(
  (gat_node_1): GATv2Conv(25, 128, heads=10)
  (mincut1): MinCut(
    (conv1): GCNConv(1280, 128)
    (pool1): Linear(in_features=128, out_features=1021, bias=True)
    (conv2): DenseGraphConv(128, 128)
    (pool2): Linear(in_features=128, out_features=511, bias=True)
    (conv3): DenseGraphConv(128, 128)
    (lin1): Linear(in_features=128, out_features=32, bias=True)
  )
  (graphnorm1): GraphNorm(32)
  (lin): Linear(in_features=32, out_features=2, bias=True)
)


In [ ]:
import warnings
warnings.filterwarnings('ignore')

def train():
    model.train()
    for data in train_loader:  # Iterate in batches over the training dataset.
         data = data.to(device)
         data.y = data.y.type(torch.LongTensor).to(device)
         optimizer.zero_grad()  # Clear gradients.

         out = model(data.x, data.edge_index, data.edge_attr, data.batch) # GAT
         #out, mc_loss, o_loss = model(data.x.type(torch.float), data.edge_index, data.batch) # MinCutPool
         #out = model(data.x.type(torch.float), data.edge_index, data.batch) # DiffPool
         
         loss = criterion(out, data.y)  # Compute the loss.
         #loss = F.nll_loss(out, data.y.view(-1)) + mc_loss + o_loss
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.


def test(loader):
     model.eval()
        
     metrics = {"accuracy":0,
               "precision":0,
               "recall":0,
               "f1_score":0}

     correct = 0
     actual, predicted = list(), list()
     for data in loader:  # Iterate in batches over the training/test dataset.
         data = data.to(device)
         data.y = data.y.type(torch.LongTensor).to(device)

         out = model(data.x, data.edge_index, data.edge_attr, data.batch) # GAT
         #out, mc_loss, o_loss = model(data.x.type(torch.float), data.edge_index, data.batch) # MinCutPool
         #out = model(data.x.type(torch.float), data.edge_index, data.batch) # DiffPool

         pred = out.cpu().argmax(dim=1).tolist()  # Use the class with highest probability.
         true = data.y.cpu().tolist()
         correct += sum(x == y for x, y in zip(true, pred)) # Check against ground-truth labels.
         actual += true
         predicted += pred 

     metrics['accuracy'] = correct / len(loader.dataset)
     metrics['precision'], metrics['recall'], metrics['f1_score'], _ = precision_recall_fscore_support(actual, predicted, average = 'binary') # precision, recall, f1score
     
     return metrics  # Derive ratio of correct predictions.

epochs = 100
for epoch in range(epochs):
    train()
    train_metrics = test(train_loader)
    test_metrics = test(test_loader)

    #print(train_metrics)
    #if epoch%10==0  or epoch==epochs:
    print(f'Epoch: {epoch:03d} '
          f'Tr Acc: {train_metrics["accuracy"]:.3f}, Tr Rec: {train_metrics["recall"]:.3f}, Tr Pre: {train_metrics["precision"]:.3f}, Tr F1: {train_metrics["f1_score"]:.3f} '
          f'Te Acc: {test_metrics["accuracy"]:.3f}, Te Rec: {test_metrics["recall"]:.3f}, Te Pre: {test_metrics["precision"]:.3f}, Te F1: {test_metrics["f1_score"]:.3f}')

Epoch: 000 Tr Acc: 0.740, Tr Rec: 0.690, Tr Pre: 0.766, Tr F1: 0.726 Te Acc: 0.736, Te Rec: 0.683, Te Pre: 0.784, Te F1: 0.730
Epoch: 001 Tr Acc: 0.770, Tr Rec: 0.785, Tr Pre: 0.761, Tr F1: 0.772 Te Acc: 0.774, Te Rec: 0.783, Te Pre: 0.785, Te F1: 0.784
Epoch: 002 Tr Acc: 0.783, Tr Rec: 0.777, Tr Pre: 0.785, Tr F1: 0.781 Te Acc: 0.782, Te Rec: 0.774, Te Pre: 0.802, Te F1: 0.788
Epoch: 003 Tr Acc: 0.780, Tr Rec: 0.802, Tr Pre: 0.768, Tr F1: 0.784 Te Acc: 0.782, Te Rec: 0.803, Te Pre: 0.786, Te F1: 0.794
Epoch: 004 Tr Acc: 0.785, Tr Rec: 0.787, Tr Pre: 0.783, Tr F1: 0.785 Te Acc: 0.791, Te Rec: 0.789, Te Pre: 0.807, Te F1: 0.798
Epoch: 005 Tr Acc: 0.788, Tr Rec: 0.794, Tr Pre: 0.784, Tr F1: 0.789 Te Acc: 0.792, Te Rec: 0.797, Te Pre: 0.804, Te F1: 0.800
Epoch: 006 Tr Acc: 0.784, Tr Rec: 0.820, Tr Pre: 0.764, Tr F1: 0.791 Te Acc: 0.791, Te Rec: 0.828, Te Pre: 0.785, Te F1: 0.806


KeyboardInterrupt: ignored